In [1]:
from __future__ import print_function
from logging import handlers
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from log import Logger
import re
import time
import os.path
import requests
import csv
import traceback
import log
import config
global loggingLevel,csvName
global GWS
GWS=[]

def match():
    allGWSprojects =[
        ["Google Workspace for Education Standard","101031","1010310005"],
        ["Google Workspace for Education Standard (Staff)","101031","1010310006"],
        ["Google Workspace for Education Standard (Extra Student)","101031","1010310007"],
        ["Google Workspace for Education Plus","101031","1010310008"],
        ["Google Workspace for Education Plus (Staff)","101031","1010310009"],
        ["Google Workspace for Education Plus (Extra Student)","101031","1010310010"],
        ["Google Workspace for Education: Teaching and Learning Upgrade","101037","1010370001"],                
                    ]
    for i in range(len(config.wantTodeleteGWS)):
        for j in range(len(allGWSprojects)):
            if config.wantTodeleteGWS[i] == allGWSprojects[j][0]:
                GWS.append([allGWSprojects[j][0],allGWSprojects[j][1],allGWSprojects[j][2]])


#讀csv檔
def readThedata():
    log.logger.debug('開始讀csv檔')
    if os.path.isfile(config.csvname) ==True:
        # changeList:要更改的list
        global changeList
        changeList =[]
        # total:要更改的總人數
        global total
        total = 0
        # incorrectEmail:錯誤email的list
        global incorrectEmail
        incorrectEmail =[]
        with open(config.csvname,newline='') as csvfile:
            #讀取 CSV 檔案內容
            log.logger.debug('開始讀取CSV 檔案內容')
            # 逐行讀取內容
            rows = csv.reader(csvfile)
            #以迴圈輸出每一列
            for row in rows:
                s1 = ''.join(row)
#               判段是否為電子郵件格式(xxxx@xxx.xx)
                # ^開頭 是一個或多個字母、數字、底線或連字符
                # 最後是 . 開頭的多個 (字母、數字、底線或連字符)
                if re.search('^[a-zA-Z0-9_-]+@[a-zA-Z0-9_-]+(\.[a-zA-Z0-9_-]+)+$', s1):
                    changeList.append(row[0])
                else:
#                     Email Address [Required]為第一列的標題，所以不放進錯誤内
                    if(row[0] !="Email Address [Required]"):
#                         log檔WARN
                        incorrectEmail.append(row[0])
                        log.logger.warning("Email格式錯誤:{0}".format(s1))
        log.logger.debug('讀取CSV 檔案完成')

        # total:要更改的總人數
        total = len(changeList)
        if total!=0:
            log.logger.info("總計 {0}筆Google Workspace 帳戶待處理".format(total))        
            print("總計 {0}筆Google Workspace 帳戶待處理".format(total))   
            return True
        else:
            log.logger.warning("總計0筆資料要處理")
            return False

    else:
        # 讀不到正確的檔案
        log.logger.error('{0}檔案不存在'.format(csvName))
        print('{0}檔案不存在'.format(csvName))

        return False
#OAuth2.0連線建立
def OAuth2():
    #OAuth2.0
    log.logger.debug('開始建立OAuth2.0連線')
    # 用try except來捕捉異常
    try:
        SCOPES = ['https://www.googleapis.com/auth/apps.licensing']
        global creds
        creds = None
        if os.path.exists('token.json'):
            #token.json存在
            log.logger.debug('token存在')
            creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)
            with open('token.json', 'w') as token:
#                 建立一個放憑證的token
                log.logger.debug('建立一個放憑證的token')
                token.write(creds.to_json())
#                 建立token完成
                log.logger.debug('建立token完成')

        log.logger.debug('建立OAuth2.0連線成功')
        return True
    except Exception :
#         error
        log.logger.error('OAuth2.0失敗')
        e = traceback.format_exc()
        log.logger.error(e)
        print('OAuth2.0失敗')
#         程式結束

        return False
#   建立服務 & 移除授權         
def deleteLicensing():
    log.logger.debug('開始建立licensing服務')       
    try:      
        service = build('licensing', 'v1', credentials=creds)
        log.logger.debug('licensing服務建立成功')
        
    except Exception :
        print('licensing服務建立失敗') 
        log.logger.error('licensing服務建立失敗')
        # 收集錯誤的原因
        e = traceback.format_exc()
        log.logger.error(e) 
        return False
        
        
    try:    
    #   依序移除授權
        log.logger.debug('開始依序移除授權') 
        global complete
        complete = 0
        global noAnylicense
        noAnylicense=[]
        global changeErrorlist
        changeErrorlist =[]
        
        for i in range(total):
#             判斷每個帳號是否要轉換
            needTodelete =False              
            for j in range(len(GWS)):
                try:
                    service.licenseAssignments().get(productId=GWS[j][1], skuId=GWS[j][2], userId=changeList[i]).execute()
#                   有授權
                    log.logger.debug('使用者[{0}]有{1}的授權'.format(changeList[i],GWS[j][0]))
                    try:
                        log.logger.debug('使用者[{0}]開始移除{1}的授權'.format(changeList[i],GWS[j][0]))
                        service.licenseAssignments().delete(productId=GWS[j][1], skuId=GWS[j][2], userId=changeList[i]).execute()
                        needTodelete =True
                        log.logger.info('使用者[{0}]移除{1}的授權成功'.format(changeList[i],GWS[j][0]))
#                         刪除授權成功
          
                    except Exception :
#                     刪除授權失敗   
                        e = traceback.format_exc()
                        #取後面的資料
                        e = e.split("returned ")[-1]
#                         刪除授權失敗的帳號,授權,原因
                        changeErrorlist.append(changeList[i])
                        changeErrorlist.append(GWS[j][0])
                        changeErrorlist.append(e)
    
                        log.logger.debug('使用者[{0}]移除{1}結果[失敗]，原因[{2}]'.format(changeList[i],GWS[j][0],e))
                except Exception :
#                     e = traceback.format_exc()
#                     errorStatuscode = e.split("<HttpError ")[-1].split("when")[0]
#                     message = e.split("{'message':")[-1].split(", ")[0]
#                     沒有該授權
                    log.logger.debug('使用者[{0}]沒有{1}的授權'.format(changeList[i],GWS[j][0]))
#             該帳號沒有任何授權
            if needTodelete==False:
                noAnylicense.append(changeList[i])
                log.logger.warning('使用者[{0}]沒有任何的授權'.format(changeList[i]))
            else:
#                 只要該帳號有一個授權被刪除，即算完成
                complete = complete +1
            currentCount = i+1  
            progressBar = round((currentCount/total)*100,2)
#              處理進度 {}/220 (10%)
            print("\r 處理進度 {0}/{1} ({2}%)".format(currentCount,total,progressBar),end=" ")
        return True
    except Exception :
        print('依序移除授權失敗')        
        log.logger.error('依序移除授權失敗')
        e = traceback.format_exc()
        log.logger.error(e)
        return False
    
    
    
def endSomeerror():
        log.logger.info("程式結束")
        print("詳細處理記錄請參照 Log檔 "+logFilename)   
        print("程式結束")
        
def endNoerror():
    fail = total-complete  
    log.logger.info("\n處理完畢：  帳戶成功{0}， 帳戶失敗{1}".format(complete,fail))
    
    # 把授權失敗的存成txt檔
    failTxt = "fail_"+date+".txt"
    with open(failTxt,"w") as f:
        f.write("失敗的帳號-email格式不符 :\n")
        for i in incorrectEmail:
            f.write("{0}\n".format(i))
        f.write("失敗的帳號-刪除授權失敗 :\n")
        for i in range(len(changeErrorlist)//3):
            f.write("使用者[{0}]移除{1}失敗，原因[{2}]\n".format(changeErrorlist[i],changeErrorlist[i+1],changeErrorlist[i+2]))
        f.write("失敗的帳號-沒有任何授權 :\n")
        for i in noAnylicense:
            f.write("{0}\n".format(i))

    print("處理完畢：  帳戶成功{0}， 帳戶失敗{1}".format(complete,fail))
    print("詳細處理記錄請參照 Log檔 "+logFilename) 
    print("程式結束")
    log.logger.info("程式結束")


    
def deleteToken():
    if os.path.exists("token.json"):
        os.remove('token.json')   
        
if __name__ == '__main__':

#     在畫面上顯示 程式開始:
#     建立.log檔
    localtime = time.localtime()
    date = time.strftime("%Y-%m-%d", localtime)
    #logFilename:log檔的檔案名
    global logFilename
    logFilename = "log_"+date+".txt"
    log = Logger(logFilename,level=config.loggingLevel)
    log.logger.info('程式開始')
    print("程式開始:\n")
    match()

    if readThedata()&OAuth2()==True:
        if deleteLicensing()==True: 
            endNoerror()
        else:
            endSomeerror()            
    else:
        endSomeerror()
    deleteToken()

程式開始:

總計 39筆Google Workspace 帳戶待處理
Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=1057604333998-4uta3ckjrc3jnltfvu4shou56lssagd9.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A64896%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fapps.licensing&state=CC4YQbSBmaCP60VsoIATN42etj0baY&access_type=offline
 處理進度 39/39 (100.0%) 處理完畢：  帳戶成功0， 帳戶失敗39
詳細處理記錄請參照 Log檔 log_2022-06-27.txt
程式結束
